In [ ]:
import os
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

# Define CIFAR-10 class names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

# Data transformation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Load CIFAR-10 dataset
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False, num_workers=2)

# Define CNN model
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 12, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 24, 5)
        self.fc1 = nn.Linear(24 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)  # CIFAR-10 → 10 classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instantiate model, loss function, and optimizer
net = NeuralNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Training loop
for epoch in range(30):
    net.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1} Loss: {running_loss / len(train_loader):.4f}')

# Save model
torch.save(net.state_dict(), 'trained_net.pth')

# Reload model for evaluation
net.load_state_dict(torch.load('trained_net.pth'))
net.eval()

# Evaluate on test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'✅ Accuracy on test images: {accuracy:.2f}%')

# Image transformation for custom image prediction
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = new_transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image

# Predict on external images
image_paths = [
    '../images/image1.jpg',
    '../images/image2.jpg',
]

net.eval()
with torch.no_grad():
    for path in image_paths:
        if not os.path.exists(path):
            print(f"⚠️ Image not found: {path}")
            continue
        img_tensor = load_image(path).to(device)
        output = net(img_tensor)
        _, pred = torch.max(output, 1)
        print(f'🖼️ {os.path.basename(path)} → Predicted: {class_names[pred.item()]}')


In [ ]:
import os
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import CosineAnnealingLR

print("Starting script execution")
print("Torch version:", torch.__version__)
print("Torch installation location:", torch.__file__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

print("Defining CIFAR-100 class names")
class_names = [f"class_{i}" for i in range(100)]  # CIFAR-100 has 100 classes

print("Setting up data transformations")
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(32, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

print("Loading CIFAR-100 dataset")
train_data = torchvision.datasets.CIFAR100(root='./data', train=True, transform=train_transform, download=True)
test_data = torchvision.datasets.CIFAR100(root='./data', train=False, transform=test_transform, download=True)

print("Creating data loaders")
train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

print("Defining ResidualBlock class")
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
    
    def forward(self, x):
        print("Executing ResidualBlock forward pass")
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

print("Defining NeuralNet class")
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        print("Initializing NeuralNet layers")
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self.make_layer(64, 64, 2)
        self.layer2 = self.make_layer(64, 128, 2, stride=2)
        self.layer3 = self.make_layer(128, 256, 2, stride=2)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(256, 100)  # CIFAR-100 has 100 classes

    def make_layer(self, in_channels, out_channels, blocks, stride=1):
        print(f"Creating layer with {blocks} blocks, in_channels={in_channels}, out_channels={out_channels}, stride={stride}")
        layers = []
        layers.append(ResidualBlock(in_channels, out_channels, stride))
        for _ in range(1, blocks):
            layers.append(ResidualBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        print("Executing NeuralNet forward pass")
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

print("Instantiating model")
net = NeuralNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=0.001, weight_decay=0.01)
scheduler = CosineAnnealingLR(optimizer, T_max=50)

print("Starting training loop")
for epoch in range(50):
    net.train()
    running_loss = 0.0
    print(f"Starting epoch {epoch+1}")
    for i, (inputs, labels) in enumerate(train_loader):
        print(f"Processing batch {i+1}/{len(train_loader)} in epoch {epoch+1}")
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    scheduler.step()
    print(f'Epoch {epoch+1} Loss: {running_loss / len(train_loader):.4f}')

print("Saving model")
torch.save(net.state_dict(), 'cifar100_net.pth')

print("Reloading model for evaluation")
net.load_state_dict(torch.load('cifar100_net.pth'))
net.eval()

print("Starting evaluation on test set")
correct = 0
total = 0
with torch.no_grad():
    for i, (inputs, labels) in enumerate(test_loader):
        print(f"Evaluating batch {i+1}/{len(test_loader)}")
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on test images: {accuracy:.2f}%')

print("Setting up transform for custom image prediction")
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

def load_image(image_path):
    print(f"Loading image: {image_path}")
    image = Image.open(image_path).convert("RGB")
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image

image_paths = [
    '../images/image1.jpg',
    '../images/image2.jpg',
]

print("Starting prediction on external images")
net.eval()
with torch.no_grad():
    for path in image_paths:
        if not os.path.exists(path):
            print(f"Image not found: {path}")
            continue
        img_tensor = load_image(path).to(device)
        print(f"Predicting for {os.path.basename(path)}")
        output = net(img_tensor)
        _, pred = torch.max(output, 1)
        print(f'{os.path.basename(path)} predicted as class {pred.item()}')

print("Script execution completed")